In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [21]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from transformers import AutoformerForPrediction, AutoformerConfig
from gluonts.dataset.pandas import PandasDataset
from utils.metrics import metric
from data.snp500 import snp500_daily

# Constants
CONTEXT_WINDOW = 96
PREDICTION_WINDOW = 96

# Initialize the Autoformer model with pre-trained weights
model = AutoformerForPrediction.from_pretrained("huggingface/autoformer-tourism-monthly")

# Prepare the data - using raw Close prices
sp500_d = snp500_daily.reset_index()[["Date", "Close"]]

def generate_forecast(context_data):
    """Generate and process forecast using Autoformer"""
    import torch
    from transformers import AutoformerConfig

    # Ensure data is 1D
    context = np.asarray(context_data).flatten()

    # Fetch required lengths and sizes from the model config
    config: AutoformerConfig = model.config
    lags_sequence = config.lags_sequence
    context_length = config.context_length
    input_size = config.input_size

    required_context_length = context_length + max(lags_sequence)

    if len(context) < required_context_length:
        raise ValueError(
            f"Context length {len(context)} is less than required length {required_context_length}."
        )

    # Slice context to required length
    context = context[-required_context_length:]

    # Generate time features for past context
    dates = pd.date_range(end=pd.Timestamp.now(), periods=required_context_length, freq="D")
    day_of_week = dates.dayofweek / 6.0  # Normalize
    month = (dates.month - 1) / 11.0  # Normalize
    past_time_features = np.stack([day_of_week, month], axis=-1)

    # Pad features to match input_size
    if past_time_features.shape[-1] < input_size:
        padding = input_size - past_time_features.shape[-1]
        past_time_features = np.pad(
            past_time_features, ((0, 0), (0, padding)), mode="constant"
        )

    # Generate future time features
    future_dates = pd.date_range(start=dates[-1] + pd.Timedelta(days=1), periods=PREDICTION_WINDOW, freq="D")
    future_day_of_week = future_dates.dayofweek / 6.0
    future_month = (future_dates.month - 1) / 11.0
    future_time_features = np.stack([future_day_of_week, future_month], axis=-1)

    if future_time_features.shape[-1] < input_size:
        padding = input_size - future_time_features.shape[-1]
        future_time_features = np.pad(
            future_time_features, ((0, 0), (0, padding)), mode="constant"
        )

    # Add batch dimension
    past_values = np.expand_dims(context, axis=0)
    past_time_features = np.expand_dims(past_time_features, axis=0)
    future_time_features = np.expand_dims(future_time_features, axis=0)

    # Convert to tensors
    past_values_tensor = torch.tensor(past_values, dtype=torch.float32)
    past_time_features_tensor = torch.tensor(past_time_features, dtype=torch.float32)
    future_time_features_tensor = torch.tensor(future_time_features, dtype=torch.float32)

    # Debug tensor shapes
    print("Shapes before model.generate():")
    print(f"past_values_tensor: {past_values_tensor.shape}")
    print(f"past_time_features_tensor: {past_time_features_tensor.shape}")
    print(f"future_time_features_tensor: {future_time_features_tensor.shape}")

    # Generate forecast
    forecast_output = model.generate(
        past_values=past_values_tensor,
        past_time_features=past_time_features_tensor,
        future_time_features=future_time_features_tensor,
    )

    # Extract forecast samples
    forecast_samples = forecast_output.sequences.squeeze(0).detach().numpy()

    # Extract quantiles
    low, median, high = np.quantile(forecast_samples, [0.1, 0.5, 0.9], axis=0)
    return low, median, high




def calculate_metrics(actual, predicted, insample=None):
    """Calculate all metrics using the metrics.py implementations"""
    actual = np.array(actual).flatten()
    predicted = np.array(predicted).flatten()

    mae, mse, rmse, mape, mspe = metric(predicted, actual)
    smape = 200 * np.mean(
        np.abs(predicted - actual) / (np.abs(predicted) + np.abs(actual))
    )
    mase = np.nan
    if insample is not None:
        naive_forecast = insample[:-1]
        naive_target = insample[1:]
        naive_mae = np.mean(np.abs(naive_target - naive_forecast))
        mase = mae / naive_mae if naive_mae != 0 else np.nan

    return {
        "MAE": mae,
        "MSE": mse,
        "RMSE": rmse,
        "MAPE": mape * 100,
        "SMAPE": smape,
        "MASE": mase if not np.isnan(mase) else None,
    }

def plot_forecast(data, context_window, prediction_window, median_forecast, low_forecast, high_forecast, title):
    """Create visualization with zoomed context"""
    plt.figure(figsize=(12, 6))
    last_context_point = data["Close"].iloc[-prediction_window - 1]
    median_forecast = np.insert(median_forecast, 0, last_context_point)
    low_forecast = np.insert(low_forecast, 0, last_context_point)
    high_forecast = np.insert(high_forecast, 0, last_context_point)
    forecast_dates = pd.date_range(start=data["Date"].iloc[-prediction_window], periods=len(median_forecast))
    actual_prices = np.insert(data["Close"].iloc[-prediction_window:].values, 0, last_context_point)

    plt.plot(data["Date"].iloc[-(context_window + prediction_window):-prediction_window], 
             data["Close"].iloc[-(context_window + prediction_window):-prediction_window], color="blue", label="Historical Data")
    plt.plot(forecast_dates, actual_prices, color="green", label="Actual Prices")
    plt.plot(forecast_dates, median_forecast, color="red", label="Median Forecast")
    plt.fill_between(forecast_dates, low_forecast, high_forecast, color="orange", alpha=0.3, label="80% Prediction Interval")
    plt.xlabel("Date")
    plt.ylabel("S&P 500 Price")
    plt.title(title)
    plt.legend()
    plt.grid()
    plt.show()

# Generate full period forecast
context = sp500_d["Close"].values[-CONTEXT_WINDOW:]
low, median, high = generate_forecast(context)

# Calculate metrics
actual_prices = sp500_d["Close"].values[-PREDICTION_WINDOW:]
insample_data = sp500_d["Close"].values[-CONTEXT_WINDOW - PREDICTION_WINDOW : -PREDICTION_WINDOW]
metrics = calculate_metrics(actual_prices, median, insample=insample_data)

print("\nMetrics:")
for metric_name, value in metrics.items():
    if value is not None:
        print(f"{metric_name}: {value:.4f}")
    else:
        print(f"{metric_name}: N/A")

# Plot the forecast
plot_forecast(
    sp500_d,
    CONTEXT_WINDOW,
    PREDICTION_WINDOW,
    median,
    low,
    high,
    "S&P 500 Price Prediction with Autoformer",
)

/opt/anaconda3/lib/python3.12/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Shapes before model.generate():
past_values_tensor: torch.Size([1, 61])
past_time_features_tensor: torch.Size([1, 61, 2])
future_time_features_tensor: torch.Size([1, 96, 2])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (24x20 and 22x64)